**Task 07: Querying RDF(s)**

In [24]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [25]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Nca83222d11f743a3939e13b5d8f7807c (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [36]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
print("RDFLib Format")
for s,p,o in g.triples((None,RDFS.subClassOf,ns.Person)):
  print(s,p,o)
print("SPARQL Format")
query1 = prepareQuery('''
SELECT ?subclasses
WHERE {
  ?subclasses rdfs:subClassOf ns:Person.
}
''',initNs = {"rdfs": RDFS,"ns":ns}
)
for r in g.query(query1):
  print(r.subclasses)

RDFLib Format
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
SPARQL Format
http://somewhere#Researcher


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [42]:
print("RDFLib Format")
for s,p,o in g.triples((None,RDF.type,ns.Person)):
  print(s)
for s,p,o in g.triples((None,RDFS.subClassOf,ns.Person)):
  for s2,p2,o2 in g.triples((None,RDF.type,s)):
    print(s2)
print("SPARQL Format")
query2 = prepareQuery('''
SELECT ?subjects
WHERE {
  {
  ?subjects rdf:type ns:Person.
} UNION {
  ?subclass rdfs:subClassOf ns:Person.
  ?subjects rdf:type ?subclass
  }
}
''',initNs = {"rdfs": RDFS,"rdf":RDF,"ns":ns}
)
for r in g.query(query2):
  print(r)

RDFLib Format
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
SPARQL Format
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [46]:
print("RDFLib Format")
for s,p,o in g.triples((None,RDF.type,ns.Person)):
  for s2,p2,o2 in g.triples((s,None,None)):
    print(s2,p2,o2)
for s,p,o in g.triples((None,RDFS.subClassOf,ns.Person)):
  for s1,p1,o1 in g.triples((None,RDF.type,s)):
    for s2,p2,o2 in g.triples((s1,None,None)):
      print(s2,p2,o2)

print("SPARQL Format")
query3 = prepareQuery('''
SELECT ?s ?p ?o
WHERE {
  {
  ?s rdf:type ns:Person.
  ?s ?p ?o
} UNION {
  ?s rdf:type ?s1.
  ?s ?o ?p.
  ?s1 rdfs:subClassOf ns:Person
  }
}
''',initNs = {"rdfs": RDFS,"rdf":RDF,"ns":ns}
)
for r in g.query(query3):
  print(r)

RDFLib Format
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w